In [1]:
import json
import numpy
import pandas
from pprint import pprint
pandas.options.display.max_colwidth = -1

In [2]:
# runs in about 5 minutes
with open('events_with_derived.json','r') as f:
    allgames = json.load(f)

allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')

print('done')

done


In [3]:
info_df = pandas.DataFrame.from_records(allgames_df['info'], index=allgames_df.index)

column_list = ['visteam', 'hometeam', 'date', 'number', 'starttime', 'daynight']
column_list += ['site', 'timeofgame', 'attendance']
column_list += ['temp', 'fieldcond', 'precip', 'sky', 'winddir', 'windspeed']
column_list += ['umphome', 'ump1b', 'umb2b', 'umb3b', 'umplf', 'umprf']
column_list += ['wp', 'lp', 'save']
column_list += ['how_scored', 'oscorer', 'pitches', 'usedh']

#info_df.columns = column_list

#number -> gameno
#timeofgame -> length

info_df['attendance'] = pandas.to_numeric(info_df['attendance'], errors='coerce')
info_df['date'] = pandas.to_datetime(info_df['date'], errors='coerce')
info_df['number'] = pandas.to_numeric(info_df['number'], errors='coerce')
info_df['starttime'] = pandas.to_datetime(info_df['starttime'], format='%l%M%P', errors='coerce')
info_df['temp'] = pandas.to_numeric(info_df['temp'], errors='coerce')
info_df['timeofgame'] = pandas.to_timedelta(info_df['timeofgame'].astype(str) + ' minutes', unit='m', errors='coerce')
info_df = info_df.fillna('')

info_df.to_csv('gameinfo_from_events.csv')
print('Done')

Done
